In [0]:
%run ../utils/exceptions/

In [0]:
import requests
import time

In [0]:
class CoinGeckoAPI_DAO:
    def __init__(self):
        """
        Classe de acesso à API do CoinGecko
        
        Args:
            base_url (str): URL base da API
            headers (str): Cabeçalhos da requisição
        """
        self.base_url = "https://api.coingecko.com/api/v3"
        self.headers = {
            "Accept": "application/json"
        }
    
    def _validate_response(self, response):
        """
        Valida a resposta de uma requisição à API.

        Args:
            response (requests.Response): Objeto de resposta da requisição HTTP.

        Returns:
            dict | list: Conteúdo em JSON da resposta, caso o status seja 200 e haja dados.

        Raises:
            EmptyResponseError: Se o status for 200 mas a resposta estiver vazia.
            NotFoundError: Se o status for 404 (endpoint não encontrado).
            RateLimitError: Se o status for 429 (limite de requisições excedido).
            ServerError: Se o status for >= 500 (erro no servidor da API).
            HTTPError: Para outros códigos de status não tratados.
        """

        status_code = response.status_code

        match status_code:
            case 200:
                data = response.json()
                if data:
                    return data
                raise EmptyResponseError("Response vazio da API.")
            case 404:
                raise NotFoundError("Endpoint não encontrado.")
            case 429:
                raise RateLimitError("Requisições acima do rate limit - Aguarde e tente novamente.")
            case code if code >= 500:
                raise ServerError(f"Erro no servidor da API: {code}")
            case _:
                raise HTTPError(f"Erro HTTP: {status_code}")


    def make_request(self, endpoint, params=None):
        """
        Faz a requisição com validação e retry automático.

        Args:
            endpoint (str): Endpoint da API.
            params (dict): Parâmetros da requisiçao.

        Returns:
            dict | list: Dados da API
        """

        url = f'{self.base_url}/{endpoint}'

        for attempt in range(3):
            try:
                print(f'Requisição: {endpoint} (tentativa {attempt + 1})')

                response = requests.get(
                    url = url,
                    headers = self.headers,
                    params = params,
                    timeout = 30
                )

                data = self._validate_response(response)
                print(f'Sucesso: {len(data) if isinstance(data, list) else 1} registros')
                return data

            except AttemptExceeded as e:
                print(f'Tentativa {attempt + 1} falhou: {e}')
                if attempt < 2:
                    time.sleep(5)
                else:
                    print(f'Todas as tentativas falharam para {endpoint}')
                    return None